# Preparation

In [1]:
import os
import cv2
import shutil
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Video

In [2]:
DATASET_PATH = "/kaggle/input/snooker-balls/balls"
OUTPUT_PATH = "/kaggle/working"

# Dataset 

### Original Dataset Structure
Dataset structure is given in next format:
```
snooker-balls/balls/
    train/
        {class1}/
            {id_1}.jpg
            {id_2}.jpg
            ...
        {class2}/
            ...
        ...
    test/
    ...
```

### Yolo Dataset Structure
Dataset structure should be transformed to next format:
```
yolo_dataset/
    images/
        train/
            {class1}_{id_1}.jpg
            {class1}_{id_2}.jpg
            ...
        val/
            ...
    labels/
        train/
            {class1}_{id_1}.txt
            {class1}_{id_2}.txt
            ...
        val/
            ...
```

In [3]:
# root directory to save dataset in yolo format
root_dir=os.path.join(OUTPUT_PATH,"yolo_dataset")
os.makedirs(root_dir, exist_ok=True)

# train and test subdirectories with image directory
images_dir=os.path.join(root_dir,"images")
os.makedirs(images_dir, exist_ok=True)
os.makedirs(images_dir+"/train", exist_ok=True)
os.makedirs(images_dir+"/val", exist_ok=True)

# train and test subdirectories with label directory
labels_dir=os.path.join(root_dir,"labels")
os.makedirs(labels_dir, exist_ok=True)
os.makedirs(labels_dir+"/train", exist_ok=True)
os.makedirs(labels_dir+"/val", exist_ok=True)

In [4]:
# variables to convert label/id to id/label
label2id = {"black": 0, "blue": 1, "brown": 2, "green": 3, "pink": 4, "red": 5, "white": 6, "yellow": 7}
id2label = {v: k for k, v in label2id.items()}

In [5]:
for layer1 in os.listdir(DATASET_PATH):
    for layer2 in os.listdir(f'{DATASET_PATH}/{layer1}'):
        for layer3 in os.listdir(f'{DATASET_PATH}/{layer1}/{layer2}'):
            if layer3.endswith('.jpg'):
                shutil.copyfile(f'{DATASET_PATH}/{layer1}/{layer2}/{layer3}',f'{images_dir}/{layer1}/{layer2}_{layer3}'.replace('test', 'val' ))
                f = open(f'{labels_dir}/{layer1}/{layer2}_{layer3}'[:-4].replace('test', 'val' )+'.txt', 'w')
                f.write(str(label2id[layer2])+' 0.5 0.5 1 1')
                f.close()

# YOLOv8

In [6]:
!pip install ultralytics
!yolo checks
from ultralytics import YOLO

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/98/44/71231f2da4fb4a602d0cef2071adb708199e571ef89ed4a136f59c19d733/ultralytics-8.0.231-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.2/663.2 kB 26.4 MB/s eta 0:00:00
Ultralytics YOLOv8.0.231 🚀 Python-3.10.12 torch-2.0.0+cpu CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5310.5/8062.4 GB disk)

OS                  Linux-5.15.133+-x86_64-with-glibc2.31
Environment         Kaggle
Python              3.10.12
Install             pip
RAM                 31.36 GB
CPU                 Intel Xeon 2.20GHz
CUDA                None

matplotlib          ✅ 3.7.4>=3.3.0
numpy               ✅ 1.24.3>=1.22.2
opencv-python       ✅ 4.8.1.78>=4.6.0
pillow              ✅ 9.5.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.31.0>=2.23.0
scipy               ✅ 1.11.4>=1.4.1
torch               ✅ 2.0.0+cpu>=1.8.0
torchvision         ✅ 0.15.1+cpu>=0.9.0
t

### Convert Dataset to Yolo Format

In [7]:
names_content = "\n".join([f"  {label_id}: {label}" for label, label_id in label2id.items()])
dataset_content = f"""
path: "{root_dir}/"
train: "images/train"
val: "images/val"
names:
{names_content}
"""
with open(os.path.join(OUTPUT_PATH, "custom_dataset.yaml"), "w") as f:
    f.write(dataset_content)

### Train the Model

In [13]:
# pretrained model: yolov8n、yolov8s、yolov8m、yolov8l、yolov8x
model = YOLO('yolov8n-p6.yaml').load('yolov8n.pt')

# Train the model using the processed dataset
results = model.train(
    data=os.path.join(OUTPUT_PATH,'custom_dataset.yaml'),
    project='snooker_project',
    exist_ok=True,
    epochs=20,
    batch=16,
    imgsz=16,
    optimizer='Adam',
    lr0=0.001,
    lrf=0.0005
)


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    221568  ultralytics.nn.modules.conv.Conv             [128

train: Scanning /kaggle/working/yolo_dataset/labels/train.cache... 11510 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11510/11510 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 2873 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2873/2873 [00:00<?, ?it/s]

Plotting labels to snooker_project/train/labels.jpg... 


...

                   all       2873       2873      0.992      0.992      0.993      0.993



20 epochs completed in 2.649 hours.
Optimizer stripped from snooker_project/train/weights/last.pt, 9.8MB
Optimizer stripped from snooker_project/train/weights/best.pt, 9.8MB

Validating snooker_project/train/weights/best.pt...
Ultralytics YOLOv8.0.231 🚀 Python-3.10.12 torch-2.0.0+cpu CPU (Intel Xeon 2.20GHz)
YOLOv8n-p6 summary (fused): 220 layers, 4777584 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 90/90 [00:23<00:00,  3.89it/s]


                   all       2873       2873      0.993      0.993      0.993      0.993
                 black       2873        359          1      0.982       0.99       0.99
                  blue       2873        323      0.981          1      0.993      0.993
                 brown       2873        331      0.995      0.997      0.995      0.995
                 green       2873        239      0.999          1      0.995      0.995
                  pink       2873         42      0.993          1      0.995      0.995
                   red       2873        322      0.984      0.967      0.989      0.989
                 white       2873       1233          1      0.997      0.995      0.995
                yellow       2873         24      0.994          1      0.995      0.995
Speed: 0.0ms preprocess, 5.2ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to snooker_project/train


lr/pg0,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▁▁▂▂▂▁█▇▇▇▆▆▅▅▄▄▃▃▂▂
lr/pg2,▁▁▂▂▂▁█▇▇▇▆▆▅▅▄▄▃▃▂▂
metrics/mAP50(B),▁▃▄▄▅▅▇█████████████
metrics/mAP50-95(B),▁▂▃▄▄▄▇█████████████
metrics/precision(B),▃▁▁▄▆▄▇█▇▇██████████
metrics/recall(B),▁▃▃▄▄▄▅▇▇▇██████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▅▄▃▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁


### Validate the Model

In [ ]:
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

### Export the Model with **ONNX**

In [ ]:
model.export(format='onnx')

### Benchmark the Model with **ONNX**

In [ ]:
from ultralytics.utils.benchmarks import benchmark
benchmark(model='snooker_project/train/weights/best.pt', imgsz=128)

### Create a Download Link

In [ ]:
%cd $OUTPUT_PATH
!zip -r snooker_project.zip snooker_project
from IPython.display import FileLink
FileLink(r'snooker_project.zip')

# Video with PyTube

In [ ]:
!pip install pytube
from pytube import YouTube
YouTube('https://youtu.be/hw02UKK4Kb0').streams.filter().get_highest_resolution().download(output_path=OUTPUT_PATH, filename='youtube.mp4')

In [ ]:
%cd $OUTPUT_PATH
!ffmpeg -i youtube.mp4 -vcodec copy -acodec copy -ss 00:01:05 -to 00:01:15 video.mp4 -y

In [ ]:
%cd $OUTPUT_PATH
Video('video.mp4', width=840)

In [14]:
#video = os.path.join(OUTPUT_PATH, 'video.mp4')
video = '/kaggle/input/snooker-test-video/video.mp4'

# Predict the Video

In [16]:
%cd $OUTPUT_PATH
video_model = YOLO("snooker_project/train/weights/best.pt")
video_model.predict(source=video, show=False, save=True, name='predict', exist_ok=True)

/kaggle/working


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/226) /kaggle/input/snooker-test-video/video.mp4: 64x64 1 white, 22.1ms
video 1/1 (2/226) /kaggle/input/snooker-test-video/video.mp4: 64x64 1 white, 15.1ms
video 1/1 (3/226) /kaggle/input/snooker-test-video/video.mp4: 64x64 1 white, 12.4ms
video 1/1 (4/226) /kaggle/input/snooker-test-video/video.mp4: 64x64 1 white, 12.4ms
video 1/1 (5/226) /kaggle/input/snooker-test-video/video.mp4: 64x64 1 green, 1 white, 11.9ms
video 1/1 (

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'black', 1: 'blue', 2: 'brown', 3: 'green', 4: 'pink', 5: 'red', 6: 'white', 7: 'yellow'}
 orig_img: array([[[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  4,  0],
         [ 0,  4,  0],
         [ 0,  4,  0]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  4,  0],
         [ 0,  4,  0],
         [ 0,  4,  0]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  4,  0],
         [ 0,  4,  0],
         [ 0,  4,  0]],
 
        ...,
 
        [[84, 90, 85],
         [84, 90, 85],
         [84, 90, 85],
         ...,
         [90, 96, 91],
         [90, 96, 91],
         [90, 96, 91]],
 
        [[84, 90, 85],
         [84, 90, 85],
         [84, 90, 85],
         ...,
         [90, 96, 91],
       

In [17]:
%cd $OUTPUT_PATH
!ffmpeg -y -loglevel panic -i runs/detect/predict/video.avi predict_video.mp4
# Display the video 
Video("predict_video.mp4", width=840)

/kaggle/working


# Track the Video with **ByteTrack**

In [18]:
%cd $OUTPUT_PATH
videl_model = YOLO('snooker_project/train/weights/best.pt')
videl_model.track(source=video, tracker="bytetrack.yaml", save=True, name='track', exist_ok=True)

/kaggle/working
requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
  Obtaining dependency information for lapx>=0.5.2 from https://files.pythonhosted.org/packages/b1/1c/1fd887004a4d68b9694e5cfe38cf450758dcf7d7436fbe576f7a5b9293c2/lapx-0.5.5-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.5 MB/s eta 0:00:00a 0:00:01

requirements: AutoUpdate success ✅ 14.5s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'black', 1: 'blue', 2: 'brown', 3: 'green', 4: 'pink', 5: 'red', 6: 'white', 7: 'yellow'}
 orig_img: array([[[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  4,  0],
         [ 0,  4,  0],
         [ 0,  4,  0]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  4,  0],
         [ 0,  4,  0],
         [ 0,  4,  0]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  4,  0],
         [ 0,  4,  0],
         [ 0,  4,  0]],
 
        ...,
 
        [[84, 90, 85],
         [84, 90, 85],
         [84, 90, 85],
         ...,
         [90, 96, 91],
         [90, 96, 91],
         [90, 96, 91]],
 
        [[84, 90, 85],
         [84, 90, 85],
         [84, 90, 85],
         ...,
         [90, 96, 91],
       

In [19]:
%cd $OUTPUT_PATH
!ffmpeg -y -loglevel panic -i runs/detect/track/video.avi track_video.mp4
# Display the video 
Video("track_video.mp4", width=840)

/kaggle/working
